# Imports

In [31]:
import csv
import os
import re

import spacy

On importe les librairies :
* os est seulement ici au cas où on voudrait se retrouver dans l'arborescence des dossiers.
* spacy pour les tests de NLP.
* re pour aider au preprocessing.

# Instanciation du modèle de langue pour spaCy

On utilise le modèle [fr_core_news_lg](https://spacy.io/models/fr#fr_core_news_lg).

In [12]:
nlp = spacy.load("fr_core_news_lg")

On affiche les metadonnées du modèle de langue.

In [13]:
nlp.meta

{'lang': 'fr',
 'name': 'core_news_lg',
 'version': '3.0.0',
 'spacy_version': '>=3.0.0,<3.1.0',
 'description': 'French pipeline optimized for CPU. Components: tok2vec, morphologizer, parser, senter, ner, attribute_ruler, lemmatizer.',
 'author': 'Explosion',
 'email': 'contact@explosion.ai',
 'url': 'https://explosion.ai',
 'license': 'LGPL-LR',
 'spacy_git_version': 'e7db07a0b',
 'vectors': {'width': 300,
  'vectors': 500000,
  'keys': 500000,
  'name': 'fr_vectors'},
 'labels': {'tok2vec': [],
  'morphologizer': ['POS=PROPN',
   'Gender=Fem|Number=Sing|POS=DET|PronType=Dem',
   'Gender=Fem|Number=Sing|POS=NOUN',
   'Number=Plur|POS=PRON|Person=1',
   'Mood=Ind|Number=Sing|POS=VERB|Person=3|Tense=Pres|VerbForm=Fin',
   'POS=SCONJ',
   'POS=ADP',
   'Definite=Def|Gender=Masc|Number=Sing|POS=DET|PronType=Art',
   'NumType=Ord|POS=ADJ',
   'Gender=Masc|Number=Sing|POS=NOUN',
   'POS=PUNCT',
   'Gender=Masc|Number=Sing|POS=PROPN',
   'Number=Plur|POS=ADJ',
   'Gender=Masc|Number=Plur|PO

Quatre entités nommées sont reconnues par ce modèle : 
* LOC
* MISC
* ORG
* PER

Le premier objectif consistera à essayer de détecter en priorité les localisations ('LOC'), les organisations ('ORG') et les personnes ('PER').

# Analyse d'un échantillon propre (transcription manuelle)

On ouvre un fichier échantillon issu d'un document du projet LECTAUREP transcrit manuellement, et dont la segmentation a été modifiée : seule la colonne centrale a été gardée.

In [23]:
with open("../../corpus_test/lectaurep/spaCy_bac_a_sable/sample_doc28_central_column.txt", 'r', encoding='utf-8') as file:
    sample = file.read()

In [15]:
sample

'An 1901 , mois de Février\nDupuis, (par Pierre) à Paris, rue Turgot 4, à Catherine Bizial, sa femme\n- d° - (par Mme) s. n. à son mari\nMurel (par Paul Louis Georges) dt à Paris, Bd Rochechouart 68, et Marie Joséphine\nLemaire, dt à Paris, même adresse (Séparation de biens)\nDubosc (ct titre de 380+ de rente 3% au nom  de : Debuisson, Justine Adèle,\nVve de Jules) et autres, à Paris, rue de l’Arcade 20\nRouy (par Edmond Narcisse), dt à Paris, rue Cavalotti 15, et autres, pr toucher &amp; recevoir\nChabrol (par Marie Louise Vatonne, Ve de Marie Eusèbe Maxime) dt à Billancourt, route de \nVersailles, 128, à Eugène Vatrin et Marceline Freling, dt à Billancourt, rue Nationale 31, de 21 303,88\nGauthier (par Marie Alexandrine) à Paris, rue St Ferdinand 2, à ses père et mère\nCusson (par Georges Ernest Léon) dt à Paris, rue Lamarck 144, à son père\nLecrosnier (ct livret de caisse d’Epargne de Paris, de 157,19 au nom de Pierre\nAuguste) décédé en son domicile à Paris, rue Descartes 21, le 16

On le normalise :

* Suppression des \n
* Ajout d'un espace avant les majuscules, pour essayer de faciliter l'analyse et la segmentation des mots. Cela ne marchera pas partout, mais permettra de mettre en évidence les noms propres dans un premier temps.

In [16]:
# On utilise une regex pour ajouter un espace avant les majuscules
sample_norm = re.sub(r"(\w)([A-Z])", r"\1 \2", sample)

In [17]:
sample_norm

'An 1901 , mois de Février\nDupuis, (par Pierre) à Paris, rue Turgot 4, à Catherine Bizial, sa femme\n- d° - (par Mme) s. n. à son mari\nMurel (par Paul Louis Georges) dt à Paris, Bd Rochechouart 68, et Marie Joséphine\nLemaire, dt à Paris, même adresse (Séparation de biens)\nDubosc (ct titre de 380+ de rente 3% au nom  de : Debuisson, Justine Adèle,\nVve de Jules) et autres, à Paris, rue de l’Arcade 20\nRouy (par Edmond Narcisse), dt à Paris, rue Cavalotti 15, et autres, pr toucher &amp; recevoir\nChabrol (par Marie Louise Vatonne, Ve de Marie Eusèbe Maxime) dt à Billancourt, route de \nVersailles, 128, à Eugène Vatrin et Marceline Freling, dt à Billancourt, rue Nationale 31, de 21 303,88\nGauthier (par Marie Alexandrine) à Paris, rue St Ferdinand 2, à ses père et mère\nCusson (par Georges Ernest Léon) dt à Paris, rue Lamarck 144, à son père\nLecrosnier (ct livret de caisse d’Epargne de Paris, de 157,19 au nom de Pierre\nAuguste) décédé en son domicile à Paris, rue Descartes 21, le 16

On instancie l'objet nlp avec notre échantillon. Cet objet est une pipeline de NLP dont les différentes étape peuvent être affichées ainsi : 

In [18]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x7f392000e590>),
 ('morphologizer',
  <spacy.pipeline.morphologizer.Morphologizer at 0x7f391ffcad10>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x7f39200331c0>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7f3920093a60>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x7f39200bab40>),
 ('lemmatizer', <spacy.lang.fr.lemmatizer.FrenchLemmatizer at 0x7f39200bf900>)]

In [19]:
doc = nlp(sample_norm)

On observe comment le modèle segmente les phrases (non-existantes) du texte, ou plutôt comment il essaye de redonner une structure logique.

In [20]:
for sent in doc.sents:
    print(sent.text)
    print("\n")

An 1901 , mois de Février
Dupuis, (par Pierre) à Paris, rue Turgot 4, à Catherine Bizial, sa femme
- d° - (par Mme)


s.


n. à son mari
Murel (par Paul Louis Georges) dt à Paris, Bd


Rochechouart 68, et Marie Joséphine
Lemaire, dt à Paris, même adresse (


Séparation de biens)



Dubosc (ct titre de 380+ de rente 3% au nom  de : Debuisson, Justine Adèle,
Vve de Jules) et autres, à Paris, rue de l’Arcade 20



Rouy (par Edmond Narcisse), dt à Paris, rue Cavalotti 15, et autres, pr toucher &amp;


recevoir
Chabrol (par Marie Louise Vatonne, Ve de Marie Eusèbe Maxime) dt à Billancourt, route de 
Versailles, 128, à Eugène Vatrin et Marceline Freling, dt à Billancourt, rue Nationale 31, de 21 303,88



Gauthier (par Marie Alexandrine) à Paris, rue St Ferdinand 2, à ses père et mère
Cusson (par Georges Ernest Léon) dt à Paris, rue Lamarck 144, à son père
Lecrosnier (ct livret de caisse d’Epargne de Paris, de 157,19 au nom de Pierre
Auguste) décédé en son domicile à Paris, rue


Descartes



On utilise displacy pour avoir une représentation des POS de l'échantillon, toujours pour voir s'il y a une structure syntaxique exploitable.

In [21]:
# Import the displacy package
from spacy import displacy

# Add some options to display it nicely
options = {"compact": True, "distance": 200, "color": "green"}

# Pass the excert document to displacy to display
displacy.render(doc, options=options, jupyter=True)

D'emblée, problème apparent : difficulté de la reconstruction syntaxique à cause de la récupération de l'intégralité de la transcription, sans distinction des colonnes du document originel.

On affiche les entités récupérées automatiquement dans le texte sous la forme suivante :

`token: label de l'entité`

In [22]:
for entity in doc.ents:
    print(f'{entity.text}: {entity.label_}')

An 1901: MISC
Février
Dupuis: ORG
Pierre: MISC
Paris: LOC
rue Turgot: LOC
Catherine Bizial: PER
Mme: PER
Murel: MISC
Paul Louis Georges: PER
Paris: LOC
Marie Joséphine
Lemaire: PER
Paris: LOC
Séparation: MISC
Dubosc: PER
Debuisson: PER
Justine Adèle: PER
Vve de Jules: PER
Paris: LOC
rue de l’Arcade 20
: LOC
Rouy: LOC
Edmond Narcisse: PER
Paris: LOC
Cavalotti 15: MISC
Chabrol: PER
Marie Louise Vatonne: PER
Ve: LOC
Marie Eusèbe Maxime: PER
Billancourt: LOC
Versailles: LOC
Eugène Vatrin: PER
Marceline Freling: PER
Billancourt: LOC
rue Nationale 31: LOC
Gauthier: PER
Marie Alexandrine: PER
Paris: LOC
rue St Ferdinand 2: LOC
Cusson: PER
Georges Ernest Léon: PER
Paris: LOC
Lamarck: PER
Lecrosnier: PER
Epargne de Paris: MISC
Pierre
Auguste: PER
Paris: LOC
Descartes: PER
Louis Emile: PER
Villeneuve-St Georges: MISC
Dupont: PER
Léonie Albertine Octavie: PER
rue St: LOC
Lazare: LOC
Honoré Bouscarat: PER
Montelaux: LOC
Aveyron: LOC
Achille Dareau: PER
Guyot: PER
Lecanta y Mendieta: PER
Joaquina D

# Rule-based matching

Il est possible de récupérer des entités non détectées en constituant des règles. Nous essayons cette solution sur un autre échantillon transcrit manuellement dans le cadre du projet Lectaurep, dont la segmentation n'a pas été modifié : toute la page a été gardée.

Nous constituerons ces règles à partir des référentiels fournis par les Archives Nationals :
* Mots matières
* Types documents

In [25]:
with open("../../corpus_test/lectaurep/spaCy_bac_a_sable/sample_doc53_central_column.txt", 'r', encoding='utf-8') as file:
    sample_2 = file.read()

In [26]:
sample_without_breaks_2 = sample_2.replace('\n', ' ')
# On utilise une regex pour ajouter un espace avant les majuscules
sample_norm_2 = re.sub(r"(\w)([A-Z])", r"\1 \2", sample_without_breaks_2)

In [27]:
doc_2 = nlp(sample_norm_2)

On commence par récupérer les mots matières.

In [29]:
with open("../../corpus_test/lectaurep/spaCy_bac_a_sable/referentiels/FRAN_RI_004_motsMatieres.tsv", 'r') as file:
    referentiel_mots_matieres = csv.reader(file, delimiter="\t")
    # On récupère la colonne Terme
    mots_matieres = [i[1:2] for i in referentiel_mots_matieres]
    # On retire le header
    mots_matieres = mots_matieres[1:]
    
mots_matieres

[['1% artistique'],
 ['abattage'],
 ['abattoir'],
 ['abbaye'],
 ['absence'],
 ['abstention'],
 ['académie'],
 ["accès à l'eau"],
 ['accession à la propriété'],
 ["accident d'avion"],
 ['accident de la circulation'],
 ['accident du travail'],
 ['accident scolaire'],
 ['accord bilatéral'],
 ['accord commercial'],
 ['accord de défense'],
 ['accord économique'],
 ['accord international'],
 ['accord salarial'],
 ["accords d'Evian (1962)"],
 ['acheminement postal'],
 ['acier'],
 ['Acte additionnel de 1815'],
 ['acte administratif'],
 ['Acte unique européen (1986)'],
 ['action concertée'],
 ['action culturelle'],
 ['action humanitaire'],
 ['actionnaire'],
 ['actionnariat salarié'],
 ['action publique'],
 ['action sociale'],
 ['action thématique programmée'],
 ['activité parlementaire'],
 ['activité périscolaire'],
 ['activité socio-éducative'],
 ["adduction d'eau"],
 ['adjudication'],
 ['administration après 1789'],
 ['administration centrale'],
 ['administration pénitentiaire'],
 ['adolescen

On récupère ensuite les types documents.

In [30]:
with open("../../corpus_test/lectaurep/spaCy_bac_a_sable/referentiels/FRAN_RI_001_typesDocuments.tsv", 'r') as file:
    referentiel_mots_matieres = csv.reader(file, delimiter="\t")
    # On récupère la colonne Terme
    types_documents = [i[1:2] for i in referentiel_mots_matieres]
    # On retire le header
    types_documents = types_documents[1:]
    
types_documents

[['accord'],
 ['accord bilatéral'],
 ['accord commercial'],
 ['accord de travail'],
 ['accord économique'],
 ['accord international'],
 ["acte d'état civil"],
 ['acte de société'],
 ['acte de vente'],
 ['acte imparfait'],
 ['acte notarié'],
 ['actes de congrès'],
 ['acte sous seing privé'],
 ['affiche'],
 ['agenda'],
 ['album'],
 ['annonce légale'],
 ['annuaire'],
 ["appel d'offres"],
 ['armorial'],
 ['arrêt'],
 ['arrêté'],
 ['article de presse'],
 ['atlas'],
 ['attestation'],
 ["autorisation d'exercice professionnel"],
 ["autorisation d'urbanisme"],
 ['autorisation de mise sur le marché'],
 ['avant-projet'],
 ["avant-projet d'architecture"],
 ['avant-projet de loi'],
 ['avis'],
 ['bail'],
 ['base de données'],
 ['bibliographie'],
 ['bilan'],
 ['bilan comptable'],
 ["brevet d'invention"],
 ['brevet notarié'],
 ['brochure'],
 ['budget'],
 ['bulle pontificale'],
 ['bulletin de salaire'],
 ['bulletin de vote'],
 ["cahier d'enchères"],
 ['cahier de doléances'],
 ['cahier des charges'],
 ['

On importe le matcher spacy.

In [32]:
from spacy.matcher import PhraseMatcher
from spacy.matcher import Matcher
from spacy.tokens import Span

On instancie l'objet PhraseMatcher avec l'objet vocab. Ce dernier stocke le vocabulaire d'un modèle chargé.

In [33]:
matcher_types_documents = PhraseMatcher(nlp.vocab)

## Création du matcher pour les types documents

In [34]:
# On fait passer tous les mots récupérés par la pipeline nlp() et on stocke la liste
phrase_patterns_types_documents = [nlp(text) for i in types_documents for text in i]

# On applique les patterns au matcher object
matcher_types_documents.add('typesdocuments', None, *phrase_patterns_types_documents)

On récupère les potentiels matches.

In [35]:
matches_types_documents = matcher_types_documents(doc_2)

matches_types_documents
# match_id, start, end

[(11869306371800119346, 473, 474), (11869306371800119346, 1009, 1010)]

On affiche les matches récupérés.

In [36]:
for match_id, start, end in matches_types_documents:
    span = Span(doc_2, start, end, label=match_id)
    print(span.text, span.label_)

procuration typesdocuments
donation typesdocuments


## Création du matcher pour les mots matières

In [37]:
matcher_mots_matieres = PhraseMatcher(nlp.vocab)

In [38]:
phrase_patterns_mots_matieres = [nlp(text) for i in mots_matieres for text in i]

# On applique les patterns au matcher object
matcher_mots_matieres.add('motsmatieres', None, *phrase_patterns_mots_matieres)

In [39]:
matches_mots_matieres = matcher_mots_matieres(doc_2)

matches_mots_matieres

[(2578879220600662430, 121, 122),
 (2578879220600662430, 169, 170),
 (2578879220600662430, 204, 205),
 (2578879220600662430, 286, 287),
 (2578879220600662430, 375, 376),
 (2578879220600662430, 495, 496),
 (2578879220600662430, 503, 504),
 (2578879220600662430, 816, 817),
 (2578879220600662430, 885, 886),
 (2578879220600662430, 915, 916)]

In [40]:
for match_id, start, end in matches_mots_matieres:
    span = Span(doc_2, start, end, label=match_id)
    print(span.text, span.label_)

adjudication motsmatieres
prêt motsmatieres
succession motsmatieres
or motsmatieres
décès motsmatieres
boulevard motsmatieres
droit motsmatieres
mariage motsmatieres
change motsmatieres
publication motsmatieres


**Danger : le référentiel entraîne l'identification de boulevard comme mot matière et non comme 'LOC'**

## On recommence la procédure en excluant les matches_mots_matieres

In [41]:
matches_types_documents = matcher_types_documents(doc_2)

for match_id, start, end in matches_types_documents:
    span = Span(doc_2, start, end, label=match_id)
    print(span.text, span.label_)

procuration typesdocuments
donation typesdocuments


On crée des entités nommées pour les matches types documents.

In [42]:
# On se calque sur la liste des tags d'entités nommées, on utilise LAW
LAW = doc_2.vocab.strings[u'LAW']

# On crée les nouvelles entités en utilisant la position des tokens et en utilisant le label qu'on a créé
new_ents = [Span(doc_2, match[1], match[2], label=LAW) for match in matches_types_documents]

# On ajoute les nouvelles entités aux entités trouvées par défaut
doc_2.ents = list(doc_2.ents) + new_ents

In [43]:
for entity in doc_2.ents:
    print(f'{entity.text}: {entity.label_}')

An 1908: MISC
Mai: LOC
Louise Amable Anaïs: PER
45 Boulevard: LOC
Certificatdevie Barry: MISC
Jenny Antoinette Hélène: PER
Ternes: LOC
Barry: PER
Jeanne Marie Pierrette Suzanne: PER
Barry: PER
Jeanne Madeleine Anaïs Hélène: PER
Goyon: PER
Marie Joseph Verteuil François Xavier: PER
Londres: LOC
Angleterre: LOC
X XX Street: LOC
Wilmès: PER
Emile Léon: PER
Epinay Procès-verbal d'adjudication 7: MISC
Seine: LOC
Marcel: PER
Santeuil: PER
Berthe Pinson: PER
Réalisation de prêt Delamaire: MISC
Créditfoncier: LOC
Mr: PER
Made: PER
Etat: LOC
Charmet: ORG
Paul Louis: PER
Lucie 8 Noëla Sylvestre: MISC
M. Charmet: PER
Pr: PER
Verb: ORG
Charmet: LOC
Jeanne Van de Weyer: PER
Ecoles: LOC
Baptiste Van: PER
Aléide Leroy: PER
Paris: LOC
Neuilly s/ Marne: LOC
Notoriété Faivre: PER
Auguste Robert: PER
rue des Roses: LOC
Dijon: LOC
Côte d'or: LOC
Bail Février: MISC
Barbe Bir: LOC
Chamos Elysées: LOC
Perreux 488: LOC
Seine: LOC
Alfred: PER
Louis Honoré Prudhomme: PER
Juliette Augustine Gruchon: PER
Bd de Be

# Modification des règles pour segmentation des phrases

Précédemment, nous avons remarqué que la segmentation ne pouvait pas être proprement réalisée à cause de la nature des documents.

SpaCy propose deux façons de segmenter les phrases d'un texte : 
* Une segmentation statistique, réalisée dans la pipeline spaCy par le **parser**, qui traite les dépendances.
* Une segmentation des phrases à l'aide de règles.

Nous explorerons cette deuxième solution pour cette expérimentation.

In [44]:
nlp_sentence_segmentation_new_rule = spacy.load("fr_core_news_lg")

On importe le même document utilisé au début de ce notebook, mais cette fois on utilise une version manuellement annotée après récupération de la transcription manuelle. 

On a ainsi rajouté des **;** à chaque fin d'unité sémantique, correspondant donc à une entrée faite par un.e notaire. Nous nous baserons donc sur les points-virgules pour segmenter le texte et voir si cela a un impact sur le NER.

Le choix du point-virgule est pour le moment temporaire. Celui-ci n'est en effet pas robuste, les notaires utilisant ce symbole de ponctuation.

In [45]:
with open('../../corpus_test/lectaurep/doc_28_sample/manual_transcription/artificial_sentence_segmentation/text_with_artificial_separation.txt', 'r', encoding='UTF-8') as file:
    sample_artificial_separation = file.read()

On ajoute à la pipeline le sentencizer.

In [46]:
# On crée une variable config qui va stocker la configuration du sentencizer.
# La variable prend la forme d'un dictionnaire, contenant une clé punt_chars et un dictionnaire en valeur.
config = {"punct_chars": [";"]}

# On ajoute le sentencizer dans la pipeline, avec la config, et on s'assure qu'on fait exécuter cette tâche avant le parser.
nlp_sentence_segmentation_new_rule.add_pipe("sentencizer", config=config, before="parser")

In [47]:
nlp_sentence_segmentation_new_rule.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x7f39162279f0>),
 ('morphologizer',
  <spacy.pipeline.morphologizer.Morphologizer at 0x7f39161bbae0>),
 ('sentencizer', <spacy.pipeline.sentencizer.Sentencizer at 0x7f391628d640>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x7f3916298fa0>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7f391628ae20>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x7f39161a5640>),
 ('lemmatizer', <spacy.lang.fr.lemmatizer.FrenchLemmatizer at 0x7f39161a80c0>)]

In [48]:
doc_3 = nlp_sentence_segmentation_new_rule(sample_artificial_separation)

In [49]:
for sent in doc_3.sents:
    print(sent)
    print("\n")

An 1901 , mois de Février ;



Dupuis, (par Pierre) à Paris, rue Turgot 4, à Catherine Bizial, sa femme ;



- d° - (par Mme) s. n. à son mari ;



Murel (par Paul Louis Georges) dt à Paris, Bd Rochechouart 68, et Marie Joséphine
Lemaire, dt à Paris, même adresse (Séparation de biens) ;



Dubosc (ct titre de 380+ de rente 3% au nom  de : Debuisson, Justine Adèle,
Vve de Jules) et autres, à Paris, rue de l’Arcade 20 ;



Rouy (par Edmond Narcisse), dt à Paris, rue Cavalotti 15, et autres, pr toucher &amp;


recevoir ;



Chabrol (par Marie Louise Vatonne, Ve de Marie Eusèbe Maxime) dt à Billancourt, route de 
Versailles, 128, à Eugène Vatrin et Marceline Freling, dt à Billancourt, rue Nationale 31, de 21 303,88 ;



Gauthier (par Marie Alexandrine) à Paris, rue St Ferdinand 2, à ses père et mère ;



Cusson (par Georges Ernest Léon) dt à Paris, rue Lamarck 144, à son père ;



Lecrosnier (ct livret de caisse d’Epargne de Paris, de 157,19 au nom de Pierre
Auguste) décédé en son domicile

In [50]:
for sent in doc_3.sents:
    print([token.text for token in sent])

['An', '1901', ',', 'mois', 'de', 'Février', ';']
['\n', 'Dupuis', ',', '(', 'par', 'Pierre', ')', 'à', 'Paris', ',', 'rue', 'Turgot', '4', ',', 'à', 'Catherine', 'Bizial', ',', 'sa', 'femme', ';']
['\n', '-', 'd°', '-', '(', 'par', 'Mme', ')', 's.', 'n.', 'à', 'son', 'mari', ';']
['\n', 'Murel', '(', 'par', 'Paul', 'Louis', 'Georges', ')', 'dt', 'à', 'Paris', ',', 'Bd', 'Rochechouart', '68', ',', 'et', 'Marie', 'Joséphine', '\n', 'Lemaire', ',', 'dt', 'à', 'Paris', ',', 'même', 'adresse', '(', 'Séparation', 'de', 'biens', ')', ';']
['\n', 'Dubosc', '(', 'ct', 'titre', 'de', '380', '+', 'de', 'rente', '3', '%', 'au', 'nom', ' ', 'de', ':', 'Debuisson', ',', 'Justine', 'Adèle', ',', '\n', 'Vve', 'de', 'Jules', ')', 'et', 'autres', ',', 'à', 'Paris', ',', 'rue', 'de', 'l’', 'Arcade', '20', ';']
['\n', 'Rouy', '(', 'par', 'Edmond', 'Narcisse', ')', ',', 'dt', 'à', 'Paris', ',', 'rue', 'Cavalotti', '15', ',', 'et', 'autres', ',', 'pr', 'toucher', '&', 'amp', ';']
['recevoir', ';']
['\n', '

In [51]:
for entity in doc_3.ents:
    print(f'{entity.text}: {entity.label_}')

An 1901: MISC
Dupuis: ORG
Pierre: MISC
Paris: LOC
rue Turgot: LOC
Catherine Bizial: PER
Mme: PER
Murel: MISC
Paul Louis Georges: PER
Paris: LOC
Bd Rochechouart: LOC
Marie Joséphine
Lemaire: PER
Paris: LOC
Séparation: MISC
Dubosc: PER
Debuisson: PER
Justine Adèle: PER
Vve de Jules: PER
Paris: LOC
rue de l’Arcade 20: LOC
Rouy: LOC
Edmond Narcisse: PER
Paris: LOC
Cavalotti 15: MISC
Chabrol: PER
Marie Louise Vatonne: PER
Ve: LOC
Marie Eusèbe Maxime: PER
Billancourt: LOC
Versailles: LOC
Eugène Vatrin: PER
Marceline Freling: PER
Billancourt: LOC
rue Nationale 31: LOC
Gauthier: PER
Marie Alexandrine: PER
Paris: LOC
rue St Ferdinand 2: LOC
Cusson: PER
Georges Ernest Léon: PER
Paris: LOC
Lamarck: PER
Lecrosnier: PER
Epargne de Paris: MISC
Pierre
Auguste: PER
Paris: LOC
rue Descartes: LOC
Louis Emile: PER
Villeneuve-St Georges: MISC
Dupont: PER
Léonie Albertine Octavie: PER
rue St: LOC
Lazare: LOC
Honoré Bouscarat: PER
Montelaux: LOC
Aveyron: LOC
Achille Dareau: PER
Guyot: PER
Lecanta y Mendieta

In [52]:
len(doc_3.ents)

170